# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
!pip install nltk
!pip install plotly

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# import libraries
import os
import pandas as pd
import pickle
import sys
from scipy.stats import gmean
import numpy as np
from sqlalchemy import create_engine
import re

In [4]:
# import functions from sklearn pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [11]:
# reading data from sql database and storing in dataframe
database_filepath = "disaster_response_db.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"

df = pd.read_sql_table(table_name,engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#verifying to see if any column contains all zero
df.loc[:, (~df.astype(bool)).all()].columns

Index(['child_alone'], dtype='object')

In [14]:
#removing column 'child_alone' as it contains all zeros
df.drop(['child_alone'], inplace=True, axis=1)

In [13]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
# from above, we can see that the max value for 'related' column is 2.
#lets analyse further
df.groupby('related').count()

,id,message,original,genre,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
related,,,,,,,,,,,,,,,,,,,,,
0,6122,6122,3395,6122,6122,6122,6122,6122,6122,6122,...,6122,6122,6122,6122,6122,6122,6122,6122,6122,6122
1,19906,19906,6643,19906,19906,19906,19906,19906,19906,19906,...,19906,19906,19906,19906,19906,19906,19906,19906,19906,19906
2,188,188,132,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188


In [18]:
# looks like 2 could be due to error. Hence replacing 2 with 1
df['related'] = df['related'].map(lambda r: 1 if r==2 else r)  

In [19]:
# getting data for modelling 
#input data 
X = df['message']
#target data
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [26]:
def tokenize(input_text):
    """
    Tokenizing text
    
    Arguments:
        text -> text messages passed as input
    Output:
        tokenized_text -> list of tokens extracted from the text is returned
    """
    
    # Lets replace all urls with urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urlplaceholder = 'urlplaceholder'
    
    # find urls from given text 
    urls_identified = re.findall(url_regex, input_text)
    
    # replacing urls with placeholders
    for urls in urls_identified:
        input_text = input_text.replace(urls, urlplaceholder)

    # using nltk word_tokenize lets extract the word tokens from the given text
    tokens = nltk.word_tokenize(input_text)
    
    # using nltk lammatizer, covert the tokens to its verb form
    lemmatizer = nltk.WordNetLemmatizer()

    # getting a list of tokenized text
    tokens_tokenized = [lemmatizer.lemmatize(x).lower().strip() for x in tokens]
    
    return tokens_tokenized

In [27]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [28]:
pipeline1 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline1.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [30]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.87      4940
               request       0.77      0.54      0.64      1124
                 offer       0.00      0.00      0.00        29
           aid_related       0.74      0.59      0.66      2661
          medical_help       0.64      0.27      0.38       517
      medical_products       0.57      0.27      0.37       312
     search_and_rescue       0.63      0.23      0.33       168
              security       0.53      0.07      0.12       116
              military       0.56      0.35      0.43       182
                 water       0.75      0.63      0.68       428
                  food       0.79      0.68      0.73       730
               shelter       0.78      0.51      0.62       587
              clothing       0.71      0.45      0.55       101
                 money       0.56      0.31      0.40       143
        missing_people       0.61      

In [31]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.95      0.89     15154
               request       0.80      0.54      0.65      3350
                 offer       0.52      0.15      0.23        89
           aid_related       0.77      0.62      0.68      8199
          medical_help       0.65      0.28      0.40      1567
      medical_products       0.72      0.35      0.47      1001
     search_and_rescue       0.66      0.22      0.33       556
              security       0.49      0.06      0.10       355
              military       0.68      0.36      0.47       678
                 water       0.78      0.65      0.71      1244
                  food       0.82      0.68      0.75      2193
               shelter       0.80      0.54      0.65      1727
              clothing       0.76      0.42      0.54       304
                 money       0.62      0.31      0.41       461
        missing_people       0.72    

### 6. Improve your model
Use grid search to find better parameters. 

In [32]:
'''parameters = 

cv =''' 

# pipeline1.get_params().keys()
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

#parameters_grid = {'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline1, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__learning_rate': [0.01, 0.02, 0.05], 'classifier__estimator__n_estimators': [10, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
# Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

In [34]:
# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.75      1.00      0.86      4940
               request       0.62      0.39      0.48      1124
                 offer       0.00      0.00      0.00        29
           aid_related       0.79      0.19      0.31      2661
          medical_help       0.65      0.12      0.20       517
      medical_products       0.65      0.10      0.17       312
     search_and_rescue       0.62      0.17      0.27       168
              security       0.00      0.00      0.00       116
              military       0.00      0.00      0.00       182
                 water       0.60      0.85      0.70       428
                  food       0.76      0.66      0.71       730
               shelter       0.86      0.30      0.45       587
              clothing       0.65      0.28      0.39       101
                 money       0.00      0.00      0.00       143
        missing_people       0.72      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [35]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.77      1.00      0.87     15154
               request       0.61      0.38      0.47      3350
                 offer       1.00      0.02      0.04        89
           aid_related       0.81      0.19      0.31      8199
          medical_help       0.63      0.11      0.19      1567
      medical_products       0.75      0.14      0.23      1001
     search_and_rescue       0.64      0.20      0.30       556
              security       0.00      0.00      0.00       355
              military       0.29      0.00      0.01       678
                 water       0.57      0.84      0.68      1244
                  food       0.78      0.68      0.73      2193
               shelter       0.84      0.29      0.44      1727
              clothing       0.78      0.33      0.47       304
                 money       1.00      0.01      0.02       461
        missing_people       0.66    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
#cv.best_params_

#Use pipeline2 which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline2.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.94      0.88      5016
               request       0.77      0.54      0.64      1137
                 offer       0.00      0.00      0.00        22
           aid_related       0.74      0.62      0.67      2695
          medical_help       0.56      0.27      0.37       524
      medical_products       0.65      0.29      0.40       312
     search_and_rescue       0.60      0.17      0.27       185
              security       0.22      0.07      0.10       102
              military       0.59      0.33      0.43       219
                 water       0.75      0.64      0.69       437
                  food       0.79      0.68      0.73       738
               shelter       0.80      0.56      0.66       565
              clothing       0.64      0.42      0.51       104
                 money       0.57      0.28      0.37       155
        missing_people       0.60      

In [37]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.95      0.88     15078
               request       0.79      0.55      0.65      3337
                 offer       0.50      0.07      0.13        96
           aid_related       0.76      0.61      0.68      8165
          medical_help       0.67      0.30      0.41      1560
      medical_products       0.71      0.36      0.47      1001
     search_and_rescue       0.67      0.21      0.32       539
              security       0.42      0.07      0.12       369
              military       0.67      0.35      0.46       641
                 water       0.78      0.65      0.71      1235
                  food       0.81      0.71      0.76      2185
               shelter       0.80      0.55      0.65      1749
              clothing       0.77      0.45      0.57       301
                 money       0.67      0.33      0.44       449
        missing_people       0.70    

### 9. Export your model as a pickle file

In [38]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.